In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Prompt Template
Smarter f-strings.

In [2]:
# Simple input-output LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

our_prompt = """
I love trips, and I have been to 6 countries.
I plan to visit a few more soon.

Can you create a post for tweet in 10 words for the above?
"""

print(llm.invoke(our_prompt).content)

"Travel bug bit me hard, visited 6 countries, more coming soon! #wanderlust"


In [3]:
# Using F-strings (a simplistic way of what prompt-template tries to do)
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

our_text = "I love trips, and I have been to 6 countries. I plan to visit a few more soon."
words_count = 10

our_prompt = f"""
{our_text}

Can you create a post for a tweet in {words_count} words for the above?
"""

print(llm.invoke(our_prompt).content)

"Travel enthusiast: 6 countries down, more adventures on the horizon! ✈️🌍"


In [4]:
# A prompt template example - smart, reusable f-string templates
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

template = """
{our_text}

Can you create a post for a tweet in {words_count} words for the above?
"""

prompt = PromptTemplate(
    input_variables=["our_text", "words_count"],
    template=template
)

final_prompt = prompt.format(words_count=3, our_text="I love trips, and I have been to 6 countries. I plan to visit a few more soon.")
print(llm.invoke(final_prompt).content)

"Traveling is life! ✈️🌍 #wanderlust"


# Saving and Loading Prompts

In [10]:
print("Prompt: ", prompt)
prompt.save("my_prompt.json")

Prompt:  input_variables=['our_text', 'words_count'] template='\n{our_text}\n\nCan you create a post for a tweet in {words_count} words for the above?\n'


In [12]:
from langchain.prompts import load_prompt

prompt_loaded = load_prompt("my_prompt.json")
print("Prompt: ", prompt_loaded)

Prompt:  input_variables=['our_text', 'words_count'] template='\n{our_text}\n\nCan you create a post for a tweet in {words_count} words for the above?\n'


# Example Selectors
For multiple examples (like few-shot prompting).

In [51]:
# Simple prompt example
our_prompt = """
You are a 5 year old girl, who is very funny, mischievous and sweet:

Question: What is a house?
Response: ''
"""

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=.9)
print(llm.invoke(our_prompt).content)

A house is like a giant cozy hug with a roof on top! It's where you can play hide and seek, have sleepovers with your friends, and eat lots of yummy snacks. It's like your own magical castle where you can make the rules and have adventures every day!


In [52]:
# FewShotPrompting - a way of giving your model some examples in the prompt before asking it for an answer
our_prompt = """
You are a 5 year old girl, who is very funny, mischievous and sweet:

Here are some examples:

Question: What is a mobile?
Response: A mobile is a magical device that fits in your pocket, like a mini-enchanted playground. It has games and videos.

Questions: What are your dreams?
Response: My dreams are like colorful adventures, where I become a superhero and save the day! I dream of giggles and ice-cream.

Question: What is a house?
Response: ''
"""

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=.9)
print(llm.invoke(our_prompt).content)

A house is a big cozy hug that keeps you safe from monsters under the bed. It's where you can have pillow forts and dance parties!


In [53]:
# FewShotPrompts - a structured way of providing examples in your prompts
# A dict structure for the examples
examples = [
    {
        "query": "What is a mobile?",
        "answer": "A mobile is a magical device that fits in your pocket, like a mini-enchanted playground. It has games and videos."
    },
    {
        "query": "What are your dreams?",
        "answer": "My dreams are like colorful adventures, where I become a superhero and save the day! I dream of giggles and ice-cream."
    }
]

# Template used to give the examples to the model
example_template = """
Question: {query}
Response: {answer}
"""

# Instructions that come before the examples (here can be role specification, or similar)
prefix = """
You are a five year old girl, who is very funny, mischievous and sweet:
Here are some examples:
"""

# Instructions that come after the examples (here should be the actual question)
suffix = """
Question: {user_input}
Response: ''
"""

# Specifying a prompt template (smart f-string) for the examples
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# Connecting everything for a reusable template of providing examples to prompts
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["user_input"],
    example_separator="\n\n"
)

# Try it out
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
query = few_shot_prompt_template.format(user_input="What is a house?")
print(llm.invoke(query).content)

A house is a big cozy nest where my toys and I live happily ever after. It's where I have tea parties and build forts with blankets and pillows.


### Length Based (Example Selector)
Constraint on the number of examples based on the max number of tokens.

In [55]:
# A dict structure for the examples
examples = [
    {
        "query": "What is a mobile?",
        "answer": "A mobile is a magical device that fits in your pocket, like a mini-enchanted playground. It has games and videos."
    },
    {
        "query": "What are your dreams?",
        "answer": "My dreams are like colorful adventures, where I become a superhero and save the day! I dream of giggles and ice-cream."
    }
]

# Template used to give the examples to the model
example_template = """
Question: {query}
Response: {answer}
"""

# Instructions that come before the examples (here can be role specification, or similar)
prefix = """
You are a five year old girl, who is very funny, mischievous and sweet:
Here are some examples:
"""

# Instructions that come after the examples (here should be the actual question)
suffix = """
Question: {user_input}
Response: ''
"""

# Specifying a prompt template (smart f-string) for the examples
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# Constraining the number of examples in the prompt based on MAX_TOKEN count - useful for smaller LLM costs
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50
)

# Connecting everything for a reusable template of providing examples to prompts
few_shot_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["user_input"],
    example_separator="\n\n"
)

# Try it out
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
query = few_shot_prompt_template.format(user_input="What is a house?")
print(query)
print(llm.invoke(query).content)


You are a five year old girl, who is very funny, mischievous and sweet:
Here are some examples:



Question: What is a mobile?
Response: A mobile is a magical device that fits in your pocket, like a mini-enchanted playground. It has games and videos.



Question: What is a house?
Response: ''

A house is like a giant puzzle that you get to live in! It's where you keep all your toys and have sleepovers with your friends.


In [56]:
# Adding more examples for the ExamplePrompt
new_example = {
    "query": "What is your favourite work?",
    "answer": "sleep"
}
few_shot_prompt_template.example_selector.add_example(new_example)

# Try it out
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
query = few_shot_prompt_template.format(user_input="What is a house?")
print(query)
print(llm.invoke(query).content)


You are a five year old girl, who is very funny, mischievous and sweet:
Here are some examples:



Question: What is a mobile?
Response: A mobile is a magical device that fits in your pocket, like a mini-enchanted playground. It has games and videos.



Question: What is a house?
Response: ''

A house is like a big cozy blanket that protects you from the outside world. It's where all the best snacks are kept and where you can have pillow fights with your family!


# Output Parser
For telling an LLM which output you want. Sometimes you might need only a smarter model in order to handle this.

In [41]:
# CSV

# Telling the model how you want it to format the result
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

# The prompt template (smart f-string)
prompt = PromptTemplate(
    template="Provide 5 examples of {query}.\n{format_instructions}",
    input_variable=["query"],
    partial_variables={"format_instructions": format_instructions}
)

# Try it out (does not work as good with gpt-3.5)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
query = prompt.format(query="Currencies")
print(llm.invoke(query).content)

# Try it out (smarter model works better)
llm = ChatOpenAI(model="gpt-4o")
query = prompt.format(query="Currencies")
print(llm.invoke(query).content)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
1. US Dollar
2. Euro
3. Japanese Yen
4. British Pound
5. Australian Dollar
U.S. Dollar,Euro,Japanese Yen,British Pound,Australian Dollar


In [57]:
# JSON

# To specify specific text that will be added to LLM's output (for JSON outputs) - this is not added by the model, but it is post-processed
response_schemas = [
    ResponseSchema(name="currency", description="answer to the user's question"),
    ResponseSchema(name="abbreviation", description="Whats the abbreviation of that currency")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
print(output_parser)

# Specifying the output parser (the format to add to the JSON output)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

# An example of a prompt
prompt = PromptTemplate(
    template="Answer the user's question as best as possible.\n{format_instructions}\n{query}",
    input_variable=["query"],
    partial_variables={"format_instructions": format_instructions}
)

# Try it out
llm = ChatOpenAI(model="gpt-4o")
query = prompt.format(query="What's the currency of India?")
print(llm.invoke(query).content)

response_schemas=[ResponseSchema(name='currency', description="answer to the user's question", type='string'), ResponseSchema(name='abbreviation', description='Whats the abbreviation of that currency', type='string')]
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"currency": string  // answer to the user's question
	"abbreviation": string  // Whats the abbreviation of that currency
}
```
```json
{
	"currency": "Indian Rupee",
	"abbreviation": "INR"
}
```
